In [1]:


import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split



device = torch.device('cpu')


In [2]:
# read the dataset and reduce the dataset size

train_data = pd.read_csv('train.csv')

res = []
for i in range(10):
    res.append(train_data.loc[train_data['label'] == i, :].sample(n=1000))
    
result = pd.concat(res)
result['label'].value_counts()


7    1000
6    1000
5    1000
4    1000
3    1000
2    1000
9    1000
1    1000
8    1000
0    1000
Name: label, dtype: int64

In [3]:

Y = result.label.values
X = result.drop("label", axis = 1).values.reshape((10000, 1, 28, 28)).astype("float32") / 255


x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 10, stratify = Y)

x_train, x_test = torch.from_numpy(x_train), torch.from_numpy(x_test)
y_train, y_test = torch.from_numpy(y_train), torch.from_numpy(y_test)

batch_size = 512

train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size = batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset (x_test, y_test), batch_size = batch_size)

In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(7*7*64,64)
        self.fc2 = nn.Linear(64,10)
        self.sm = nn.Softmax(dim=None)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.sm(out)
        return out

In [5]:
model = ConvNet()

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20, verbose=True)


In [7]:
# Training for 30 epochs

n_epochs = 30
model.train()

for epoch in range(n_epochs):
    train_loss = 0.0
    
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        scheduler.step(loss)
        
    print('Epoch is:', epoch)
    print('Training loss is:', train_loss/len(train_loader.dataset))

/home/sushrut/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch is: 0
Training loss is: 2.0140453033447265
Epoch is: 1
Training loss is: 1.6274664011001587
Epoch is: 2
Training loss is: 1.569475664138794
Epoch is: 3
Training loss is: 1.5502930068969727
Epoch is: 4
Training loss is: 1.538114369392395
Epoch is: 5
Training loss is: 1.5277278833389283
Epoch is: 6
Training loss is: 1.5198277416229249
Epoch is: 7
Training loss is: 1.5159213638305664
Epoch is: 8
Training loss is: 1.507336217880249
Epoch is: 9
Training loss is: 1.502874108314514
Epoch is: 10
Training loss is: 1.5006384840011597
Epoch is: 11
Training loss is: 1.495161029815674
Epoch is: 12
Training loss is: 1.4923762416839599
Epoch is: 13
Training loss is: 1.4906935577392577
Epoch   230: reducing learning rate of group 0 to 5.0000e-04.
Epoch is: 14
Training loss is: 1.4881203746795655
Epoch   251: reducing learning rate of group 0 to 2.5000e-04.
Epoch is: 15
Training loss is: 1.4846615924835205
Epoch is: 16
Training loss is: 1.483572135925293
Epoch   272: reducing learning rate of gro

In [8]:
# Testing

test_loss = 0.0
class_correct = list(0.0 for i in range(10))
class_total = list(0.0 for i in range(10))


model.eval()

for data, target in test_loader:
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item() * data.size(0)
    _, pred = torch.max(output, 1)
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    
    for i in range(len(target)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
        
test_loss = test_loss/len(test_loader.dataset)

print('Test loss:', test_loss)

print('Overall Test Accuracy:', (np.sum(class_correct)) / np.sum(class_total))

/home/sushrut/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test loss: 1.4999643449783324
Overall Test Accuracy: 0.963
